In [2]:
import anndata as ad
import pandas as pd
import numpy as np
import scanpy as sc
import decoupler as dc
import os
#import pyreadr

In [4]:
#R_input = pd.read_csv("LR2TF_test_run\\resultsCrossTalkeR_input_control.csv")

#R_input = R_input.where(R_input["MeanLR"] > 0, np.nan) 
#R_input.dropna(inplace=True)


#R_input.to_csv("R_CRT_no_negative_score_ctrl.csv")

R_input = pd.read_csv("LR2TF_test_run/resultsCrossTalkeR_input_PMF_MF2.csv")

R_input = R_input.where(R_input["MeanLR"] > 0, np.nan) 
R_input.dropna(inplace=True)


R_input.to_csv("R_CRT_no_negative_score_PMF.csv")

In [ ]:
import pyreadr
ligands_readin = pyreadr.read_r("ligands_human.rda")
ligands = ligands_readin["ligands_human"]
ligands.to_csv("ligands_human.csv", index=False)

ligands_readin_m = pyreadr.read_r("converted_ligands.rda")
print (ligands_readin_m)
ligands_m = ligands_readin_m["converted_ligands"]
ligands_m.to_csv("converted_ligands.csv", index=False)

rtf_readin = pyreadr.read_r("RTF_DB_2.rda") 
print(rtf_readin)
rtf_db = rtf_readin["RTF_DB_2"]
rtf_db.to_csv("rtf_db.csv", index=False)

rtf_m_readin = pyreadr.read_r("RTF_DB_mouse.rda") 
print (rtf_m_readin)
rtf_db_m = rtf_m_readin["RTF_DB_mouse"]
rtf_db_m.to_csv("rtf_db_mouse.csv", index=False)

In [ ]:
#chatgpt generated, prolly trash since it did not give the same output as R code

import pandas as pd
import numpy as np
import re

def add_entry(source, target, gene_A, gene_B, type_gene_A, type_gene_B, MeanLR):
    return {
        "source": source,
        "target": target,
        "gene_A": gene_A,
        "gene_B": gene_B,
        "type_gene_A": type_gene_A,
        "type_gene_B": type_gene_B,
        "MeanLR": MeanLR
    }

def generate_CrossTalkeR_input(tf_activities, gene_expression, out_path, regulon = None, organism = "human"):
    if organism == "human":
        ligands = pd.read_csv("ligands_human.csv")
        R2TF = pd.read_csv("rtf_db_human.csv")
    elif organism == "mouse": 
        ligands = pd.read_csv("ligands_mouse.csv")
        R2TF = pd.read_csv("rtf_db_mouse.csv")
    else:
        NameError("Invalid organism to generate CrossTalkeR input!")

    
    R2TF = R2TF.set_index("tf")
 
    sorted_regulon = regulon.groupby("tf")["target"].apply(list).to_dict()
    import json
    with open('file.txt', 'w') as file:
     file.write(json.dumps(sorted_regulon))
    
    tf_activities = tf_activities[tf_activities["t_value"] > 0]

    output_list = []

    for _, row in tf_activities.iterrows():
        tf = row["gene"]
        cluster = row["cluster"]
        t_value = row["t_value"]

        targets = sorted_regulon.get(tf, [])
        receptors = R2TF.get(tf, [])

        tf_ligands = np.intersect1d(targets, ligands)

        for ligand in tf_ligands:
            if ligand in gene_expression.index:
                ex_value = gene_expression.at[ligand, cluster]
                if ex_value != 0:
                    output_list.append(add_entry(
                        source=cluster, target=cluster,
                        gene_A=tf, gene_B=ligand,
                        type_gene_A="Transcription Factor", type_gene_B="Ligand",
                        MeanLR=t_value
                    ))

        for receptor in receptors:
            output_list.append(add_entry(
                source=cluster, target=cluster,
                gene_A=receptor, gene_B=tf,
                type_gene_A="Receptor", type_gene_B="Transcription Factor",
                MeanLR=t_value
            ))

    output_df = pd.DataFrame(output_list)
    output_df["gene_A"] = output_df["gene_A"].str.replace("_", "+", regex=False)
    output_df["gene_B"] = output_df["gene_B"].str.replace("_", "+", regex=False)

    return output_df


In [ ]:
def generate_CrossTalkeR_input(tf_activities, gene_expression, out_path, regulon = None):

  ligands = pd.read_csv("ligands_human.csv")
  R2TF = pd.read_csv("rtf_db.csv")

  sorted_regulon = regulon[["tf", "target"]]
  #sorted_regulon.rename(columns={"target" : "targets"})
  sorted_regulon = sorted_regulon.set_index("tf")

  R2TF = R2TF.set_index("tf")
  output_df = create_empty_CTR_dataframe()
  tf_l = create_empty_CTR_dataframe()
  r_tf = create_empty_CTR_dataframe()

  for row in range(len((tf_activities))):

    #if (tf_activities["z_score"].iloc[row] > 0):
    tf = str(tf_activities["gene"].iloc[row])
    if tf in sorted_regulon.index:
      targets = sorted_regulon.loc[tf, "target"]

    if tf in R2TF.index:
        receptors = R2TF.loc[tf, "receptor"]

    else:
        receptors = []
    tf_ligands = np.intersect1d(targets, ligands)

    if len(tf_ligands) > 0:
      for ligand in tf_ligands:
          expressed = False
          if ligand in gene_expression.index:
            ex_value = gene_expression.loc[ligand, tf_activities.iloc[row, 2]]
            if (ex_value != 0):
              expressed = True
          
          df_list_l = []
            
          if (expressed == True):
            df_list_l = add_entry_to_CTR_dataframe(source = tf_activities.iloc[row, 2],
                                                    target = tf_activities.iloc[row, 2],
                                                    gene_A =tf_activities.iloc[row, 0],
                                                    gene_B = ligand,
                                                    type_gene_A = "Transcription_Factor",
                                                    type_gene_B= "Ligand",
                                                    MeanLR= tf_activities.iloc[row, 3],
                                                    row= row)
              
            tf_l = pd.concat([tf_l, df_list_l], ignore_index=True)
            
    
      if (len(receptors) > 0):
        for receptor in receptors:
          df_list_r = [] 
          df_list_r = add_entry_to_CTR_dataframe(tf_activities.iloc[row, 2],
                                              tf_activities.iloc[row, 2],
                                              receptor,
                                              tf_activities.iloc[row, 0],
                                              "Receptor",
                                              "Transcription Factor",
                                              tf_activities.iloc[row, 3],
                                              row)
              
          print(df_list_r)
            
          r_tf = pd.concat([r_tf, df_list_r], ignore_index=True)

      #tf_l.to_csv(single_result_path + "/" + renamed_condition + "_ctr_test_l.csv", index=0)
      #r_tf.to_csv(single_result_path + "/" + renamed_condition + "_ctr_test_r.csv", index=0)

  #    r_tf["gene_A"] <- gsub("_", "+", r_tf$gene_A, fixed = TRUE)
   #   r_tf["gene_B"] <- gsub("_", "+", r_tf$gene_B, fixed = TRUE)
   #   tf_l["gene_A"] <- gsub("_", "+", tf_l$gene_A, fixed = TRUE)
   #   tf_l["gene_B"] <- gsub("_", "+", tf_l$gene_B, fixed = TRUE)

  output_df = pd.concat([output_df, r_tf], ignore_index=True)
  output_df = pd.concat([output_df, tf_l], ignore_index=True)
  #output_df.to_csv("tf_l_r.csv", index=0)


  return output_df

In [ ]:
#from get significant tf markers 


    #result1 = tf_activities_sub.uns["wilcoxon_markers"]
    #groups = result1["names"].dtype.names
    #anndataobject_markers = pd.DataFrame(
    #{group + '_' + key[:1]: result1[key][group]
    #for group in groups for key in ["names","logfoldchanges","pvals","pvals_adj"]})

    #anndataobject_markers.to_csv("rank_genes_groups_result" + renamed_condition + ".csv",index=0)
    
    #result2 = tf_activities_sub.uns["t_test_overestim_var_markers"]
    #groups = result2["names"].dtype.names
    #anndataobject_markers = pd.DataFrame(
    #{group + '_' + key[:1]: result2[key][group]
    #for group in groups for key in ["names","logfoldchanges","pvals","pvals_adj"]})

In [ ]:
def map_z_value_filtered(filtered_summarized_tf_scores_df, anndataobject_markers):
    z_score = []
    for gene in anndataobject_markers["gene"]:
        if gene in filtered_summarized_tf_scores_df.index:
            z_score.append(filtered_summarized_tf_scores_df.loc[gene, anndataobject_markers["cluster"]])
    return z_score


In [ ]:
#comparison python wilcoxon and r wilcoxon SPECIFIC MARKERS

py_result_wilcx = pd.read_csv("script_test\\TF_results\\control\\control_specific_markers_wilcoxon_test.csv")
r_result_wilcx = pd.read_csv("LR2TF_test_run\\results\\TF_results\\control\\all_specificmarker__control.csv")

r_result_wilcx = r_result_wilcx[r_result_wilcx["p_val_adj"] < 0.05]

py_result_wilcx = py_result_wilcx.rename(columns={"names": "gene", "group": "cluster"})

df_output_py = pd.DataFrame()

for i in range(len(py_result_wilcx)):
    a = py_result_wilcx["gene"].iloc[i]
    for j in range(len(r_result_wilcx)):
        b = r_result_wilcx["gene"].iloc[j]
        if a == b:
            df_output_py[i] = py_result_wilcx.iloc[i]
            

df_output_py = df_output_py.T

df_output_r = pd.DataFrame()
for i in range(len(r_result_wilcx)):
    a = r_result_wilcx["gene"].iloc[i]
    for j in range(len(py_result_wilcx)):
        b = py_result_wilcx["gene"].iloc[j]
        if a == b:
            df_output_r[i] = r_result_wilcx.iloc[i]

df_output_r = df_output_r.T

df_output = pd.merge(df_output_py, df_output_r, on="gene", suffixes=("_py", "_r"))
#df_output = py_result_wilcx.rename(columns={"gene": "gene_py", "Unnamed: 0": "gene_R"})

df_output.to_csv("tf_comparison_r_py/py_wilcoxon_r_same_genes_control_filtered.csv")

#comparison python t test and r wilcoxon

py_result = pd.read_csv("script_test\\TF_results\\control\\control_specific_markers_t_test_overestim_test.csv")
r_result = pd.read_csv("LR2TF_test_run\\results\\TF_results\\control\\all_specificmarker__control.csv")

py_result = py_result[py_result["pvals_adj"] < 0.05]
r_result = r_result[r_result["p_val_adj"] < 0.05]
py_result = py_result.rename(columns={"names": "gene", "group": "cluster"})


df_output_py = pd.DataFrame()

for i in range(len(py_result)):
    a = py_result["gene"].iloc[i]
    for j in range(len(r_result)):
        b = r_result["gene"].iloc[j]
        if a == b:
            df_output_py[i] = py_result.iloc[i]
            

df_output_py = df_output_py.T

df_output_r = pd.DataFrame()
for i in range(len(r_result)):
    a = r_result["gene"].iloc[i]
    for j in range(len(py_result)):
        b = py_result["gene"].iloc[j]
        if a == b:
            df_output_r[i] = r_result.iloc[i]

df_output_r = df_output_r.T

#df_output = pd.concat([df_output_py, df_output_r], keys= ["gene"])
df_output = pd.merge(df_output_py, df_output_r, on="gene", suffixes=("_py", "_r"))
#df_output = py_result_wilcx.rename(columns={"gene": "gene_py", "Unnamed: 0": "gene_R"})

df_output.to_csv("tf_comparison_r_py/py_t_test_r_same_genes_control_filtered.csv")

# py wilcox vs py t test
py_result = pd.read_csv("script_test\\TF_results\\control\\control_specific_markers_t_test_overestim_test.csv")
r_result = pd.read_csv("script_test\\TF_results\\control\\control_specific_markers_wilcoxon_test.csv")


py_result = py_result[py_result["pvals_adj"] < 0.05]
r_result = r_result[r_result["p_val_adj"] < 0.05]
py_result = py_result.rename(columns={"names": "gene", "group": "cluster"})


df_output_py = pd.DataFrame()

for i in range(len(py_result)):
    a = py_result["gene"].iloc[i]
    for j in range(len(r_result)):
        b = r_result["gene"].iloc[j]
        if a == b:
            df_output_py[i] = py_result.iloc[i]
            

df_output_py = df_output_py.T

df_output_r = pd.DataFrame()
for i in range(len(r_result)):
    a = r_result["gene"].iloc[i]
    for j in range(len(py_result)):
        b = py_result["gene"].iloc[j]
        if a == b:
            df_output_r[i] = r_result.iloc[i]

df_output_r = df_output_r.T

df_output = pd.merge(df_output_py, df_output_r, on="gene", suffixes=("_py", "_r"))
df_output.to_csv("tf_comparison_r_py/py_t_test_py_wilcox_same_genes_control_filtered.csv")

In [ ]:
#comparison python wilcoxon and r wilcoxon CROSSTALKER INPUT

py_result_wilcx = pd.read_csv("script_test\\CrossTalkeR_input_control.csv")
r_result_wilcx = pd.read_csv("LR2TF_test_run\\resultsCrossTalkeR_input_control.csv")


df_output_py = pd.DataFrame()

for i in range(len(py_result_wilcx)):
    a = py_result_wilcx["gene_A"].iloc[i]
    a2 = py_result_wilcx["gene_B"].iloc[i]
    a3 = py_result_wilcx["source"].iloc[i]
    a4 =  py_result_wilcx["target"].iloc[i]
    for j in range(len(r_result_wilcx)):
        b = r_result_wilcx["gene_A"].iloc[j]
        b2 = r_result_wilcx["gene_B"].iloc[j]
        b3 =  r_result_wilcx["source"].iloc[j]
        b4 =  r_result_wilcx["target"].iloc[j]
        if ((a == b) & (a2 == b2) & (a3 == b3) & (a4 == b4)):
            df_output_py[i] = py_result_wilcx.iloc[i]
            

df_output_py = df_output_py.T

df_output_r = pd.DataFrame()
for i in range(len(r_result_wilcx)):
    a = r_result_wilcx["gene_A"].iloc[i]
    a2 = r_result_wilcx["gene_B"].iloc[i]
    a3 = r_result_wilcx["source"].iloc[i]
    a4 =  r_result_wilcx["target"].iloc[i]
    for j in range(len(py_result_wilcx)):
        b = py_result_wilcx["gene_A"].iloc[j]
        b2 = py_result_wilcx["gene_B"].iloc[j]
        b3 =  py_result_wilcx["source"].iloc[j]
        b4 =  py_result_wilcx["target"].iloc[j]
        if ((a == b) & (a2 == b2) & (a3 == b3) & (a4 == b4)):
            df_output_r[i] = r_result_wilcx.iloc[i]

df_output_r = df_output_r.T

df_output = pd.merge(df_output_py, df_output_r, on="gene_A", suffixes=("_py", "_r"))
#df_output = py_result_wilcx.rename(columns={"gene": "gene_py", "Unnamed: 0": "gene_R"})

df_output.to_csv("tf_comparison_r_py/py_r_crosstalker_input.csv")



In [1]:
import pandas as pd

# Read input CSVs
py_result_wilcx = pd.read_csv("py_intra_network_ctrl.csv")
r_result_wilcx = pd.read_csv("R_intra_network_ctrl.csv")

print(len(r_result_wilcx))

# Merge DataFrames on the specified columns
common_columns = ["Receptor", "TF", "celltype", "Target_Gene"]
df_output = pd.merge(py_result_wilcx, r_result_wilcx, on=common_columns, suffixes=("_py", "_r"))

# Save the resulting DataFrame
df_output.to_csv("tf_comparison_r_py/py_r_intra_network_ctrl.csv", index=False)

3775648


In [1]:
import pandas as pd

# Read input CSVs
py_result_wilcx = pd.read_csv("py_intra_network_PMF.csv")
r_result_wilcx = pd.read_csv("R_intra_network_PMF.csv")

print(len(r_result_wilcx))
print(len(py_result_wilcx))

# Merge DataFrames on the specified columns
common_columns = ["Receptor", "TF", "celltype", "Target_Gene"]
df_output = pd.merge(py_result_wilcx, r_result_wilcx, on=common_columns, suffixes=("_py", "_r"))
print(len(df_output))

# Save the resulting DataFrame
df_output.to_csv("tf_comparison_r_py/py_r_intra_network_PMF.csv", index=False)

6765319
2341187
2623742


In [5]:
import pandas as pd

# Read input CSVs
py_result_wilcx = pd.read_csv("script_test/CrossTalkeR_input_PMF_MF2.csv")
r_result_wilcx = pd.read_csv("R_CRT_no_negative_score_PMF.csv")

# Merge DataFrames on the specified columns
common_columns = ["source", "target", "gene_A", "gene_B"]
df_output = pd.merge(py_result_wilcx, r_result_wilcx, on=common_columns, suffixes=("_py", "_r"))

# Save the resulting DataFrame
df_output.to_csv("tf_comparison_r_py/py_r_crosstalker_input_PMF_no_neg_score.csv", index=False)

In [4]:
import pandas as pd

# Read input CSVs
py_result_wilcx = pd.read_csv("script_test/CrossTalkeR_input_control.csv")
r_result_wilcx = pd.read_csv("R_CRT_no_negative_score.csv")

# Merge DataFrames on the specified columns
common_columns = ["source", "target", "gene_A", "gene_B"]
df_output = pd.merge(py_result_wilcx, r_result_wilcx, on=common_columns, suffixes=("_py", "_r"))

# Save the resulting DataFrame
df_output.to_csv("tf_comparison_r_py/py_r_crosstalker_input_control_no_neg_score.csv", index=False)

In [ ]:
#compare old R output w new R output (did scaling change results?)
import pandas as pd

# Read input CSVs
r_result_old = pd.read_csv("LR2TF_test_run/resultsCrossTalkeR_input_control.csv")
r_result_new = pd.read_csv("new_test/CrossTalkeR_input_control.csv")

# Merge DataFrames on the specified columns
common_columns = ["source", "target", "gene_A", "gene_B"]
df_output = pd.merge(r_result_old, r_result_new, on=common_columns, suffixes=("_r_old", "_r_new"))
df_output[""]

# Save the resulting DataFrame
df_output.to_csv("tf_comparison_r_py/old_r_new_r_control_condition.csv", index=False)

In [ ]:
#for cluster ctr input/single condition
import pandas as pd

# Read input CSVs
py_result_wilcx = pd.read_csv("script_test/.csv")
r_result_wilcx = pd.read_csv(".csv")

# Merge DataFrames on the specified columns
common_columns = ["source", "target", "gene_A", "gene_B"]
df_output = pd.merge(py_result_wilcx, r_result_wilcx, on=common_columns, suffixes=("_py", "_r"))

# Save the resulting DataFrame
df_output.to_csv("tf_comparison_r_py/py_r_ctr_input_ctrl_cluster.csv", index=False)

In [ ]:
#for R ctr input done w py code and R ctr input done w R code cond contr

import pandas as pd

# Read input CSVs
r_py_result_wilcx = pd.read_csv("tf_l_r_R_data_cond_contr.csv")
r_r_result_wilcx = pd.read_csv("r_output.csv")

 # Align DataFrames by filling missing columns with NaN
#r_py_result_wilcx, r_r_result_wilcx = r_py_result_wilcx.align(r_r_result_wilcx, join="outer", axis=None, fill_value=None)
    
    # Find differences
#diff = r_py_result_wilcx.compare(r_r_result_wilcx, keep_equal=False)
    
    
    # Save differences to CSV
#diff.to_csv("r_r_code_r_py_code_diff.csv", index=True)
    


#common_columns = ["source", "target", "gene_A", "gene_B", "type_gene_A", "type_gene_B", "MeanLR"]
#df_output = pd.merge(r_py_result_wilcx, r_r_result_wilcx, on=common_columns, suffixes=("_r_py", "_r_r"))

#with open('r_output.csv', 'r') as t1, open('tf_l_r_R_data_cond_contr.csv', 'r') as t2:
#    fileone = t1.readlines()
#    filetwo = t2.readlines()

#with open('r_r_code_r_py_code_diff.csv', 'w') as outFile:
#    for line in filetwo:
#        if line not in fileone:
#            outFile.write(line)

#from csv_diff import load_csv, compare
#diff = compare(
#    load_csv(open("tf_l_r_R_data_cond_contr.csv"), key="source"),
#    load_csv(open("r_output.csv"), key="source")
#)
#print(diff)
#df_output = pd.DataFrame(diff)
# Save the resulting DataFrame
#df_output.to_csv("r_r_code_r_py_code_diff.csv", index=False)

{'added': [], 'removed': [], 'changed': [{'key': 'Megakaryocyte', 'changes': {'gene_A': ['ZZZ3', 'WNK1'], 'gene_B': ['ANXA2', 'ZBTB16'], 'type_gene_A': ['Transcription_Factor', 'Receptor'], 'type_gene_B': ['Ligand', 'Transcription Factor'], 'MeanLR': ['0.531854020131922', '0.498454909350288']}}, {'key': 'Fibroblast', 'changes': {'gene_A': ['ZZZ3', 'MAPK9'], 'gene_B': ['LIFR', 'ZNF217'], 'type_gene_A': ['Transcription_Factor', 'Receptor'], 'type_gene_B': ['Ligand', 'Transcription Factor'], 'MeanLR': ['0.404718831798295', '0.435539982124441']}}, {'key': 'Neural', 'changes': {'gene_A': ['ZNF740', 'MAPK8'], 'gene_B': ['ANP32B', 'ZEB1'], 'type_gene_A': ['Transcription_Factor', 'Receptor'], 'type_gene_B': ['Ligand', 'Transcription Factor'], 'MeanLR': ['0.742719981155771', '0.104414878632952']}}], 'columns_added': [], 'columns_removed': []}


In [31]:
with open('r_output.csv', 'r') as t1:
    old_csv = t1.readlines()
with open('tf_l_r_R_data_cond_contr.csv', 'r') as t2:
    new_csv = t2.readlines()

with open('r_r_code_r_py_code_diff.csv', 'w') as out_file:
    line_in_new = 0
    line_in_old = 0
    while line_in_new < len(new_csv) and line_in_old < len(old_csv):
        if old_csv[line_in_old] != new_csv[line_in_new]:
            out_file.write(new_csv[line_in_new])
        else:
            line_in_old += 1
        line_in_new += 1

In [ ]:
#R
library(VennDiagram)

#csv1 <- read.csv("R_CRT_no_negative_score.csv")
csv1 <- read.csv("R_ctr_input_wo_exp_ctr_tables.csv")
#csv1 <- results@CTR_input_condition[["control"]]

csv1 <- csv1[c("source", "gene_A", "gene_B")]

csv1_list =list()
for (i in 1:nrow(csv1)) {
    row <- paste(csv1[i, ], collapse = ",")
    csv1_list <- append(csv1_list, row)
}

head(csv1_list)
SET1 <- csv1_list


#csv2 <- read.csv("script_test/CrossTalkeR_input_control.csv")
csv2 <- read.csv("py_ctr_input_wo_ctr_exp_tables.csv")

csv2 <- csv2[c("source", "gene_A", "gene_B")]

csv2_list =list()
for (i in 1:nrow(csv2)) {
    row <- paste(csv2[i, ], collapse = ",")
    csv2_list <- append(csv2_list, row)
}

head(csv2_list)
SET2 <- csv2_list

v <- venn.diagram(
  x = list(SET1, SET2),
  category.names = c("Set R" , "Set PY "),
  filename = NULL,
  output = TRUE)


grid.newpage()
grid.draw(v)


#v[[5]]$label  <- paste(setdiff(SET1, intersect(SET1,SET2)), collapse="\n") 
#v[[6]]$label <- paste(setdiff(SET2, intersect(SET1,SET2)), collapse="\n")


setdiff1 <- t(as.data.frame(setdiff(SET1, intersect(SET1, SET2))))
setdiff2 <- t(as.data.frame(setdiff(SET2, intersect(SET1, SET2))))

write.csv(setdiff1, "R_unique_genes_unfiltered.csv", row.names = FALSE)
write.csv(setdiff2, "Py_unique_genes_unfiltered.csv", row.names = FALSE)


grid.newpage()
grid.draw(v)

pdf("venn_diagram_PY_R_CTR_input_unfiltered.pdf")
grid.draw(v)
dev.off()


In [2]:
import pandas as pd

# Read input CSVs
r_result_wilcx = pd.read_csv("r_avg_expr_ctrl.csv")
py_result_wilcx = pd.read_csv("script_test/control_average_gene_expression_by_cluster_exp.csv")

# Merge DataFrames on the specified columns
common_columns = ["Neural", "MSC", "Fibroblast", "Megakaryocyte", "Myeloid"]
df_output = pd.merge(py_result_wilcx, r_result_wilcx, left_index=True,right_index=True, suffixes=("_py", "_r"))

# Save the resulting DataFrame
df_output.to_csv("tf_comparison_r_py/avg_gene_expr_ctrl.csv", index=False)

In [7]:
import pandas as pd

# Read input CSVs
py_result_wilcx = pd.read_csv("script_test/CrossTalkeR_input_PMF_MF2.csv")
r_result_wilcx = pd.read_csv("LR2TF_test_run/resultsCrossTalkeR_input_PMF_MF2.csv")

# Merge DataFrames on the specified columns
common_columns = ["gene_A", "gene_B"]
df_output = pd.merge(py_result_wilcx, r_result_wilcx, on=common_columns, suffixes=("_py", "_r"))

# Save the resulting DataFrame
df_output.to_csv("tf_comparison_r_py/py_r_crosstalker_input_genes_PMF.csv", index=False)

In [ ]:

    
    #  df = add_entry_to_CTR_dataframe(tf_activities.iloc[row, 2],
    #                                         tf_activities.iloc[row, 2],
    #                                         tf_activities.iloc[row, 0],
    #                                         ligand,
    #                                         'Transcription Factor',
    #                                         'Ligand',
     #                                        tf_activities.iloc[row, 3])
      
 #     

In [ ]:
gene_ids = sub_object.var.index.values
    #cluster line even necessary if not returned?
    #clusters = anndataobject.obs[celltype].cat.categories
obs = sub_object[:,gene_ids].X.toarray()
obs = np.expm1(obs)
sub_object = pd.DataFrame(obs,columns=gene_ids,index= sub_object.obs["new_annotation"])
sub_object = sub_object.groupby(level=0, observed=False).mean()
#sub_object.T.to_csv(outpath + name + "average_gene_expression_by_cluster_exp.csv")

sub_object.T


In [ ]:
anndataobject = ad.read_h5ad("LR2TF_test_run/anndata_object.h5ad")
      #  anndataobject.rename_categories("protocol", ["PMFMF2", "control"])

tf_activities = ad.read_csv("decoupler_results.csv")
tf_activities.obsm = anndataobject.obsm
tf_activities.uns = anndataobject.uns
tf_activities.obs = anndataobject.obs

tf_activities_sub = tf_activities[anndataobject.obs.protocol == "PMF,MF2"]
#print(tf_activities_sub.var.index.values)
#print(tf_activities_sub.var_names)

tf_act_2 = tf_activities_sub
tf_act_2.obs_names = tf_act_2.obs["new_annotation"]
tf_df = tf_act_2.to_df()
tf_df.groupby("new_annotation").mean().T

In [ ]:
tf_activities = ad.read_csv("decoupler_results.csv")
tf_activities.obsm = anndataobject.obsm
tf_activities.uns = anndataobject.uns
tf_activities.obs = anndataobject.obs
tf_activities = tf_activities[tf_activities.obs.protocol == "control"]
tf_activities.obs

In [ ]:
#tag mapping t over
    tag_mapping_t_over= anndataobject_markers_t_over[["gene", "tag", "log_fc_tag", "cluster", "pvals_adj", "logfoldchanges"]]

    tag_mapping_t_over = tag_mapping_t_over[(tag_mapping_t_over["pvals_adj"] < float(pval)) & 
                              ((tag_mapping_t_over["logfoldchanges"] > float(logfc)) | 
                              (tag_mapping_t_over["logfoldchanges"] < -float(logfc)))]

     # Pivot the DataFrame to create a wide format similar to `dcast` in R
    tag_mapping_t_over = tag_mapping_t_over.pivot(index="gene", columns="cluster", values="tag")
    tag_mapping_t_over.fillna("ns", inplace=True)

In [ ]:
# Summarize viper_scores_clusters by TF and cell type: compute mean and std (standard deviation)
    #summarized_viper_scores = (viper_scores_clusters
       # .groupby(['tf', 'cell_type'])  # Group by transcription factor and cell type
     #   .agg(avg=('activity', 'mean'), std=('activity', 'std'))  # Calculate mean and standard deviation
      #  .reset_index())

# Spread the summarized data to wide format: remove 'std', and spread 'tf' on columns, 'avg' as values
   # summarized_viper_scores_df = (summarized_viper_scores
   #     .drop(columns=['std'])  # Remove the std column
   #     .pivot(index='cell_type', columns='tf', values='avg')  # Spread the TFs as columns, average activity as values
    #    .reset_index(drop=True))  # Reset the index to make it a regular DataFrame

# Transpose the dataframe (same as R's t())
   # summarized_viper_scores_df = summarized_viper_scores_df.T

# Write the summarized TF scores to CSV

In [ ]:
anndataobject = ad.read_h5ad("LR2TF_test_run/anndata_object.h5ad")


tf_activities = ad.read_csv("decoupler_results.csv")
anndataobject.obsm['ulm_estimate'] = tf_activities
print(anndataobject.obsm["ulm_estimate"].X)
#for name in anndataobject.obs.protocol.unique():
##    subobject = anndataobject[anndataobject.obs.protocol == name]
#print(type(subobject)) 
#print(type(anndataobject))  
#print(subobject.X)



In [ ]:
res[i] = {}
                    for pw in pws:
                        group1 = a_sub[:, pw].X[g == vs1]
                        group2 = a_sub[:, pw].X[g == vs2]

                    
                        if len(group1) > 0 and len(group2) > 0:
                        # Wilcoxon rank-sum test
                            stat, pval = ranksums(group1, group2)

                        # Calculate effect size
                            r = abs(stat) / np.sqrt(len(group1) + len(group2))  # approximate r calculation
                        
                            res[i][pw] = {"pval": pval, "r": r}

In [ ]:
def map_z_value_filtered(filtered_summarized_tf_scores_df, anndataobject_markers):
    genes = []
    clusters = []
    z_scores = []  

    anndataobject_markers = anndataobject_markers.set_index("gene")
    #print(filtered_summarized_tf_scores_df)
    
    for i in range(len(anndataobject_markers.index)):
        a = anndataobject_markers.index[i]
        for j in range(len(filtered_summarized_tf_scores_df.index)):
            b = filtered_summarized_tf_scores_df.index[j]
            if a == b:
                c = anndataobject_markers.columns.get_loc("cluster")
                cluster = anndataobject_markers.iloc[i, c]
                gene_rows = filtered_summarized_tf_scores_df.iloc[j]
                if isinstance(gene_rows, pd.Series):
                    gene_rows = gene_rows.to_frame().T

                for _, gene_row in gene_rows.iterrows():
                     score = gene_row[cluster]
                     genes.append(a)
                     clusters.append(cluster)
                     z_scores.append(score)

    z_score_df = pd.DataFrame({
        'gene': genes,
        'cluster': clusters,
        'z_score': z_scores
    })

    return z_score_df

In [ ]:
#scipy version 7.2.25

def condition_comparison_significant(tf_activities, out_path, celltype, condition, comparison_list):
    
    tf_activities = sc.pp.scale(tf_activities, copy= True)

    vs_df_dic = {}
          
    if isinstance(comparison_list[0], str):  
        comparison_list = [comparison_list]
    
    for vs1, vs2 in comparison_list:

        print(f"vs1: {vs1}, vs2: {vs2}") 

        all_tf_list = tf_activities.var_names

        res = {}
        for i in tf_activities.obs[celltype].unique(): 
            comparison_sub = tf_activities[(tf_activities.obs[celltype] == i) & (tf_activities.obs[condition].isin([vs1, vs2]))]
            if len(pd.unique(comparison_sub.obs[condition])) == 2:
                condition_table = comparison_sub.obs[[condition]].copy()
                condition_table.columns = ["condition"]
                metadata_counts = condition_table.groupby("condition", observed = False).size()
                
                if (metadata_counts.iloc[0] + metadata_counts.iloc[1]) > 10:
                    g = comparison_sub.obs[condition].astype("category")
                    g = g.cat.set_categories([vs1, vs2])
                    
                    #scanpy
                    #sc.tl.rank_genes_groups(comparison_sub, groupby= condition,
                     #                    reference="rest", method="wilcoxon", key_added="condition_comp_markers", corr_method= "bonferroni")
                    
                    #sc.pl.rank_genes_groups_heatmap(comparison_sub, show_gene_labels=True, key="condition_comp_markers")

                    #res_tmp = sc.get.rank_genes_groups_df(comparison_sub, group = None, log2fc_min=0, key="condition_comp_markers")
                    #######

                    #decoupler
                    #comparison_sub = sc.pp.scale(comparison_sub, copy= True)
                    #res_tmp = dc.rank_sources_groups(comparison_sub, groupby= condition, reference="rest", method="wilcoxon")
                    #res_tmp.rename(columns={"group": "condition", "statistic" : "scores", "meanchange" : "logfoldchanges"}, inplace=True)
                    #####

                    res[i] = {}
                    for tf in all_tf_list:
                        group1 = comparison_sub[:, tf].X[g == vs1]
                        group2 = comparison_sub[:, tf].X[g == vs2]

                        if len(group1) > 0 and len(group2) > 0:
                            # Wilcoxon test
                            stat, pval = ranksums(group1, group2)
                            
                            r = abs(stat) / np.sqrt(len(group1) + len(group2))  
                            l2fc = np.log2((group1.mean()+ 1e-6)/(group2.mean() + 1e-6))
                            res[i][tf] = {"tf": tf,"pval": pval.item(), "r": r.item(), "logfoldchanges" : float(l2fc), "CellType" : i}

                    res[i] = pd.DataFrame.from_dict(res[i], orient="index")
                    
                    
        res_df = pd.concat(res.values(), axis=0, ignore_index=True)

        #default is Benjamini Hochberg correction
        _, res_df["FDR"], _, _ = multipletests(res_df["pval"], alpha=0.05, method='fdr_bh')
        res_df = res_df.dropna()

        def assign_significance_tag(fdr):
            if fdr < 0.001:
                return "***"
            elif fdr < 0.01:
                return "**"
            elif fdr < 0.05:
                return "*"
            else:
                return "ns"

        res_df["tag"] = res_df["FDR"].apply(assign_significance_tag)
        
        res_df.rename(columns={"names":"tf", "group": "condition"}, inplace=True)
        res_df.to_csv(f"{out_path}/all_tfs_{vs1}_vs_{vs2}.csv", index=False)

        result_name = f"{vs1}_{vs2}"
        vs_df_dic[result_name] = res_df
        #print(vs_df_dic[result_name])
    return vs_df_dic

In [ ]:
# scipy version 7.2.25

def get_significant_tfs(tf_activities_sub, condition, out_path, tf_condition_significant, celltype, pval, logfc, plot, condition_comparison = False):
    
    renamed_condition = condition.replace(",", "_")

    single_result_path = out_path + renamed_condition 
    if not os.path.isdir(single_result_path):
        os.mkdir(single_result_path)

    tf_activities_scaled = sc.pp.scale(tf_activities_sub, copy= True)
    #zero center = False is possible and delivers results for rank genes groups, but then the tf scores wont be the same as in R

    #tf_activities_sub = sc.pp.normalize_total(tf_activities_sub, copy=True)
    #tf_activities_sub = sc.pp.log1p(tf_activities_sub, copy=True) 
    # "warning: seems to be already log transformed"

    number_of_clusters = len(tf_activities_sub.obs["new_annotation"].cat.categories) 

    #decoupler
    #anndataobject_markers_wilcoxon = dc.rank_sources_groups(tf_activities_scaled, groupby= "new_annotation", reference="rest", method="wilcoxon")
    
    #scanpy
    #sc.tl.rank_genes_groups(tf_activities_sub, groupby= "new_annotation", reference="rest", method="wilcoxon", key_added="wilcoxon_markers", corr_method= "bonferroni")
    #sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="wilcoxon_markers", key_added= "wilcoxon_markers_filtered")

    #sc.tl.rank_genes_groups(tf_activities_sub, groupby= "new_annotation", reference="rest", method="t-test_overestim_var", key_added="t_test_overestim_var_markers", corr_method= "bonferroni" )
    #sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="t_test_overestim_var_markers", key_added="t_test_overestim_filtered")

    #anndataobject_markers_wilcoxon = sc.get.rank_genes_groups_df(tf_activities_sub, group = None, log2fc_min=0, key="wilcoxon_markers")
    #print(anndataobject_markers_wilcoxon)

    #scipy
    res = {}
    all_tf_list = tf_activities_sub.var_names
    for i in tf_activities_scaled.obs[celltype].unique(): 
        res[i] = {}

        group1_mask = tf_activities_sub.obs[celltype] == i
        group2_mask = tf_activities_sub.obs[celltype] != i
        
        for tf in all_tf_list:
            group1 = tf_activities_sub[group1_mask, tf].X
            group2 = tf_activities_sub[group2_mask, tf].X

            #print("g1", group1)
            #print("g2", group2)

            mean1 = np.mean(group1) if len(group1) > 0 else 0
            mean2 = np.mean(group2) if len(group2) > 0 else 0
            
            if len(group1) > 0 and len(group2) > 0:
                # Wilcoxon test
                stat, pval = ranksums(group1, group2)
                            
                r = abs(stat) / np.sqrt(len(group1) + len(group2))  
                l2fc = np.log2((group1.mean()+ 1e-6)/(group2.mean() + 1e-6))
                    
                res[i][tf] = {"gene": tf,"pval": pval.item(), "r": r.item(), "logfoldchanges" : l2fc, "cluster" : i}

        res[i] = pd.DataFrame.from_dict(res[i], orient="index")
        #print("res",res[i])
                    
                    
    anndataobject_markers_wilcoxon = pd.concat(res.values(), axis=0, ignore_index=True)
    

    #default scipy false discovery control is Benjamini Hochberg correction
    #multiple tests is what scanpy uses
    _, anndataobject_markers_wilcoxon["pvals_adj"], _, _ = multipletests(anndataobject_markers_wilcoxon["pval"], alpha=0.05, method='fdr_bh')
    anndataobject_markers_wilcoxon = anndataobject_markers_wilcoxon.dropna()


    #####################scipy end

    #anndataobject_markers_wilcoxon.rename(columns={"names": "gene", "group": "cluster", "meanchange": "logfoldchanges"}, inplace=True)

    anndataobject_markers_wilcoxon["tag"] = None
    anndataobject_markers_wilcoxon["log_fc_tag"] = None
    
    #print("after concat", anndataobject_markers_wilcoxon)
    
    anndataobject_markers_wilcoxon["tag"] = anndataobject_markers_wilcoxon["pvals_adj"].apply(eval_pval)
    anndataobject_markers_wilcoxon["log_fc_tag"] = anndataobject_markers_wilcoxon["logfoldchanges"].apply(eval_log_fc_tag)
     

    anndataobject_markers_wilcoxon.to_csv(single_result_path + "/" + renamed_condition + "_specific_markers_wilcoxon_test.csv",index=0)

    #anndataobject_markers_t_over = sc.get.rank_genes_groups_df(tf_activities_sub, group = None, log2fc_min=0, key="t_test_overestim_var_markers")
 
    #anndataobject_markers_t_over.rename(columns={"names":"gene", "group": "cluster"}, inplace=True)

    #anndataobject_markers_t_over.loc[:, "tag"] = None
    #anndataobject_markers_t_over.loc[:, "log_fc_tag"] = None

    #for i in range(len(anndataobject_markers_t_over["pvals_adj"])):
    #     anndataobject_markers_t_over.loc[i, "tag"] = eval_pval(anndataobject_markers_t_over.loc[i, "pvals_adj"])
    #     anndataobject_markers_t_over.loc[i, "log_fc_tag"] = eval_pval(anndataobject_markers_t_over.loc[i, "logfoldchanges"])

    #anndataobject_markers_t_over.to_csv(single_result_path + "/" + renamed_condition + "_specific_markers_t_test_overestim_test.csv",index=0)

   #tag mapping wilcoxon
    tag_mapping_wilcox = anndataobject_markers_wilcoxon[["gene", "tag", "log_fc_tag", "cluster", "pvals_adj", "logfoldchanges"]]
    tag_mapping_wilcox = tag_mapping_wilcox[(tag_mapping_wilcox["pvals_adj"] < float(pval))] 
    tag_mapping_wilcox = tag_mapping_wilcox[(tag_mapping_wilcox["logfoldchanges"] > float(logfc)) | 
                              (tag_mapping_wilcox["logfoldchanges"] < -float(logfc))]

    tag_mapping_wilcox = tag_mapping_wilcox.pivot(index="gene", columns="cluster", values="tag")
    clusters = anndataobject_markers_wilcoxon["cluster"].unique()

    for cluster in clusters:
        if cluster not in tag_mapping_wilcox.columns:
            tag_mapping_wilcox[cluster] = np.nan

    tag_mapping_wilcox = tag_mapping_wilcox.astype("object")
    tag_mapping_wilcox.fillna("ns", inplace=True)
    
    #makes the index of the subobject not unique --- not possible to filter by index after this
    tf_activities_scaled.obs_names = tf_activities_scaled.obs[celltype]
    tf_scores_df = tf_activities_scaled.to_df()
    tf_scores_df.columns.name = None
    unfiltered_tf_scores = create_unfiltered_tf_scores(tf_scores_df, condition, celltype, single_result_path)
   
    #Filter to only include tfs that match the tag_mapping/are markers
    #print(tag_mapping_wilcox)
    col_num = tf_scores_df.columns.isin(tag_mapping_wilcox.index)  
    #print(col_num)
    filtered_tf_scores_df = tf_scores_df.loc[:, sorted(col_num)]


    filtered_summarized_tf_scores_df = filtered_tf_scores_df.groupby(celltype, observed = False).mean().T
    filtered_summarized_tf_scores_df.index.name = "gene"
    filtered_summarized_tf_scores_df.to_csv(f"{single_result_path}/tf_scores_{condition}.csv")
    tf_scores_variable_table = save_variable_tf_score(filtered_summarized_tf_scores_df, condition, single_result_path, plot)

    if plot:
        plot_tf_activity(filtered_summarized_tf_scores_df, tag_mapping_wilcox, condition, single_result_path)
    
    filtered_summarized_tf_scores_df.index = filtered_summarized_tf_scores_df.index.map(lambda x: re.sub(".,", "_", x))
   
    filtered_summarized_tf_scores_df = filtered_summarized_tf_scores_df.where(filtered_summarized_tf_scores_df > 0, np.nan) 
    
    anndataobject_markers_wilcoxon_merged = anndataobject_markers_wilcoxon.merge(map_z_value(filtered_summarized_tf_scores_df, anndataobject_markers_wilcoxon),  on=['gene', 'cluster'], how='inner')
    anndataobject_markers_wilcoxon_merged = anndataobject_markers_wilcoxon_merged[anndataobject_markers_wilcoxon_merged.tag != "ns"]
    anndataobject_markers_wilcoxon_merged.dropna(inplace=True)
    res_wilcoxon = anndataobject_markers_wilcoxon_merged[["gene","tag", "cluster", "z_score"]]

    #anndataobject_markers_t_over_merged = anndataobject_markers_t_over.merge(map_z_value_filtered(filtered_summarized_tf_scores_df, anndataobject_markers_t_over),  on=['gene', 'cluster'], how='inner')
    #anndataobject_markers_t_over_merged = anndataobject_markers_t_over_merged[anndataobject_markers_t_over_merged.tag != "ns"]
    #anndataobject_markers_t_over_merged.dropna(inplace=True)
    #res_t_test = anndataobject_markers_t_over[["gene","tag", "cluster", "z_score"]]

    res_wilcoxon.to_csv(single_result_path + "/significant_cluster_tf_results_wilcoxon_" + renamed_condition + ".csv", index=0)
    #res_t_test.to_csv(single_result_path  + "/_significant_cluster_tf_results_t_test_" + renamed_condition + ".csv", index=0)

    res = {}
    res["cluster"] = res_wilcoxon
    
    tf_condition_significant["gene"] = tf_condition_significant["gene"].apply(lambda x: re.sub(".,", "_", x))

    if condition_comparison:
        unfiltered_tf_scores = unfiltered_tf_scores.where(unfiltered_tf_scores > 0, np.nan) 
        tf_condition_significant = tf_condition_significant.merge(map_z_value(unfiltered_tf_scores, tf_condition_significant), left_on=None, right_on=None, left_index=False, right_index=False)
        tf_condition_significant.dropna(inplace=True)
    
        res["condition"] = tf_condition_significant
        res["condition"].to_csv(f"{single_result_path}/significant_condition_tf_results_{condition}.csv", index=0)

    return res

In [ ]:
#current condition comparison (old)
def condition_comparison_significant(tf_activities, out_path, celltype, condition, comparison_list):

    vs_df_dic = {}

    if isinstance(comparison_list[0], str):  
        comparison_list = [comparison_list]
    
    for vs1, vs2 in comparison_list:

        print(f"vs1: {vs1}, vs2: {vs2}") 

        all_tf_list = tf_activities.var_names

        res = pd.DataFrame()
        for i in tf_activities.obs[celltype].unique(): 
            comparison_sub = tf_activities[(tf_activities.obs[celltype] == i) & (tf_activities.obs[condition].isin([vs1, vs2]))]
            if len(pd.unique(comparison_sub.obs[condition])) == 2:
                condition_table = comparison_sub.obs[[condition]].copy()
                condition_table.columns = ["condition"]
                metadata_counts = condition_table.groupby("condition", observed = False).size()
                
                if (metadata_counts.iloc[0] + metadata_counts.iloc[1]) > 10:
                    g = comparison_sub.obs[condition].astype("category")
                    g = g.cat.set_categories([vs1, vs2])
                    
                    #scanpy
                    #sc.tl.rank_genes_groups(comparison_sub, groupby= condition,
                     #                    reference="rest", method="wilcoxon", key_added="condition_comp_markers", corr_method= "bonferroni")
                    
                    #sc.pl.rank_genes_groups_heatmap(comparison_sub, show_gene_labels=True, key="condition_comp_markers")

                    #res_tmp = sc.get.rank_genes_groups_df(comparison_sub, group = None, log2fc_min=0, key="condition_comp_markers")
                    #######

                    #decoupler
                    comparison_sub = sc.pp.scale(comparison_sub, copy= True)
                    res_tmp = dc.rank_sources_groups(comparison_sub, groupby= condition, reference="rest", method="wilcoxon")
                    res_tmp.rename(columns={"group": "condition", "statistic" : "scores", "meanchange" : "logfoldchanges"}, inplace=True)
            
               

                    group1 = comparison_sub.X[g == vs1]
                    group2 = comparison_sub.X[g == vs2]
                        
                    
                    res_tmp["r"] = res_tmp["scores"] / np.sqrt(len(group1) + len(group2))
                    res_tmp["CellType"] = i
                    res_tmp["FDR"] = false_discovery_control(res_tmp["pvals"])
                    
                    res_tmp["logfoldchanges"] = res_tmp["logfoldchanges"].where(res_tmp["logfoldchanges"] > 0, np.nan) 

                    res = pd.concat([res, res_tmp], ignore_index=True)

        res_df = res.dropna()

        def assign_significance_tag(fdr):
            if fdr < 0.001:
                return "***"
            elif fdr < 0.01:
                return "**"
            elif fdr < 0.05:
                return "*"
            else:
                return "ns"

        res_df["tag"] = res_df["FDR"].apply(assign_significance_tag)
        
        res_df.rename(columns={"names":"tf", "group": "condition"}, inplace=True)
        res_df.to_csv(f"{out_path}/all_tfs_{vs1}_vs_{vs2}.csv", index=False)

        result_name = f"{vs1}_{vs2}"
        vs_df_dic[result_name] = res_df
        #print(vs_df_dic[result_name])
    return vs_df_dic

In [ ]:
def get_significant_tfs_single(tf_activities_sub, celltype, condition, out_path, pval, logfc, name, plot):

    renamed_condition = condition.replace(",", "_")
    name = name.replace(",", "_")

    single_result_path = out_path + renamed_condition 
    if not os.path.isdir(single_result_path):
        os.mkdir(single_result_path)

    tf_activities_scaled = sc.pp.scale(tf_activities_sub, copy= True)

    #tf_activities_sub = sc.pp.normalize_total(tf_activities_scaled, copy=True)
    #tf_activities_sub = sc.pp.log1p(tf_activities_sub, copy=True) 
    # "warning: seems to be already log transformed"

    number_of_clusters = len(tf_activities_sub.obs["new_annotation"].cat.categories) 


    sc.tl.rank_genes_groups(tf_activities_sub, groupby= "new_annotation", reference="rest", method="wilcoxon", key_added="wilcoxon_markers", corr_method= "bonferroni")
    #sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="wilcoxon_markers", key_added= "wilcoxon_markers_filtered")
    

    sc.tl.rank_genes_groups(tf_activities_sub, groupby= "new_annotation", reference="rest", method="t-test_overestim_var", key_added="t_test_overestim_var_markers", corr_method= "bonferroni" )
    #sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="t_test_overestim_var_markers", key_added="t_test_overestim_filtered")


    result1 = tf_activities_sub.uns['wilcoxon_markers']
    groups = result1['names'].dtype.names
    anndataobject_markers = pd.DataFrame(
    {group + '_' + key[:1]: result1[key][group]
    for group in groups for key in ['names','logfoldchanges','pvals','pvals_adj']})

    result2 = tf_activities_sub.uns['t_test_overestim_var_markers']
    groups = result2['names'].dtype.names
    anndataobject_markers = pd.DataFrame(
    {group + '_' + key[:1]: result2[key][group]
    for group in groups for key in ['names','logfoldchanges','pvals','pvals_adj']})

    anndataobject_markers_wilcoxon = sc.get.rank_genes_groups_df(tf_activities_sub, group = None, log2fc_min=0, key="wilcoxon_markers"
    
    
    anndataobject_markers_wilcoxon.rename(columns={"names":"gene", "group": "cluster"}, inplace=True)
    
    #documentation says this won't update the main dataframe but will only overwrite a copy in future pandas versions (after v3)
    anndataobject_markers_wilcoxon["tag"] = None
    anndataobject_markers_wilcoxon["log_fc_tag"] = None
    
    for i in range(len(anndataobject_markers_wilcoxon["pvals_adj"])):
         anndataobject_markers_wilcoxon["tag"].iloc[i,] = eval_pval(anndataobject_markers_wilcoxon["pvals_adj"].iloc[i,])
         anndataobject_markers_wilcoxon["log_fc_tag"].iloc[i,] = eval_log_fc_tag(anndataobject_markers_wilcoxon["logfoldchanges"].iloc[i,])
     

    anndataobject_markers_wilcoxon.to_csv(single_result_path + "/" + name + "_specific_markers_wilcoxon_test.csv",index=0)

    anndataobject_markers_t_over = sc.get.rank_genes_groups_df(tf_activities_sub, group = None, log2fc_min=0, key="t_test_overestim_var_markers")
 
    anndataobject_markers_t_over.rename(columns={"names":"gene", "group": "cluster"}, inplace=True)

    anndataobject_markers_t_over["tag"] = None
    anndataobject_markers_t_over["log_fc_tag"] = None
    for i in range(len(anndataobject_markers_t_over["pvals_adj"])):
         anndataobject_markers_t_over["tag"].iloc[i,] = eval_pval(anndataobject_markers_t_over["pvals_adj"].iloc[i,])
         anndataobject_markers_t_over["log_fc_tag"].iloc[i,] = eval_pval(anndataobject_markers_t_over["logfoldchanges"].iloc[i,])

    anndataobject_markers_t_over.to_csv(single_result_path + "/" + name + "_specific_markers_t_test_overestim_test.csv",index=0)

   #tag mapping wilcoxon
    tag_mapping_wilcox = anndataobject_markers_wilcoxon[["gene", "tag", "log_fc_tag", "cluster", "pvals_adj", "logfoldchanges"]]
    tag_mapping_wilcox = tag_mapping_wilcox[(tag_mapping_wilcox["pvals_adj"] < float(pval)) & 
                              ((tag_mapping_wilcox["logfoldchanges"] > float(logfc)) | 
                              (tag_mapping_wilcox["logfoldchanges"] < -float(logfc)))]

    tag_mapping_wilcox = tag_mapping_wilcox.pivot(index="gene", columns="cluster", values="tag")
    clusters = anndataobject_markers_wilcoxon["cluster"].unique()

    for cluster in clusters:
        if cluster not in tag_mapping_wilcox.columns:
            tag_mapping_wilcox[cluster] = np.nan

    tag_mapping_wilcox.fillna("ns", inplace=True)

    tf_activities_scaled.obs_names = tf_activities_scaled.obs[celltype]
    tf_scores_df = tf_activities_scaled.to_df()
    tf_scores_df.columns.name = None
    unfiltered_tf_scores = create_unfiltered_tf_scores(tf_scores_df, condition, celltype, single_result_path)
   
    #Filter to only include tfs that match the tag_mapping/are markers
    col_num = tf_scores_df.columns.isin(tag_mapping_wilcox.index)  
    filtered_tf_scores_df = tf_scores_df.loc[:, col_num] 
    
    filtered_summarized_tf_scores_df = filtered_tf_scores_df.groupby(celltype, observed = False).mean().T
    filtered_summarized_tf_scores_df.index.name = "gene"
    filtered_summarized_tf_scores_df.to_csv(f'{single_result_path}/tf_scores_{condition}.csv')
    tf_scores_variable_table = save_variable_tf_score(filtered_summarized_tf_scores_df, condition, single_result_path, plot)

    if plot:
        plot_tf_activity(filtered_summarized_tf_scores_df, tag_mapping_wilcox, condition, single_result_path)
    
    #filtered_summarized_tf_scores_df.index = re.sub((".,"), "_", filtered_summarized_tf_scores_df.index)
    anndataobject_markers_wilcoxon_merged = anndataobject_markers_wilcoxon.merge(map_z_value(filtered_summarized_tf_scores_df, anndataobject_markers_wilcoxon),  on=['gene', 'cluster'], how='inner')
    anndataobject_markers_wilcoxon_merged = anndataobject_markers_wilcoxon_merged[anndataobject_markers_wilcoxon_merged.tag != "ns"]
    anndataobject_markers_wilcoxon_merged.dropna(inplace=True)
    res_wilcoxon = anndataobject_markers_wilcoxon_merged[["gene","tag", "cluster", "z_score"]]

    #anndataobject_markers_t_over_merged = anndataobject_markers_t_over.merge(map_z_value_filtered(filtered_summarized_tf_scores_df, anndataobject_markers_t_over),  on=['gene', 'cluster'], how='inner')
    #anndataobject_markers_t_over_merged = anndataobject_markers_t_over_merged[anndataobject_markers_t_over_merged.tag != "ns"]
    #anndataobject_markers_t_over_merged.dropna(inplace=True)
    #res_t_test = anndataobject_markers_t_over[["gene","tag", "cluster", "z_score"]]

    res_wilcoxon.to_csv(single_result_path + "/" + name + "_significant_markers_wilcoxon.csv", index=0)
    #res_t_test.to_csv(single_result_path + "/" + name + "_significant_markers_t_test.csv", index=0)

     #//TODO: 
    #delete one variant and put test type as a variable

    res = {}
    res["cluster"] = res_wilcoxon
    return res

In [ ]:
#   tried to do lr table AND tf table with complex split  
    #tf_table_complex = tf_table_receptors[tf_table_receptors["gene_A"].str.contains("+", na = False, regex = False)]
    #print("tf_table_complex", tf_table_complex)
    #tf_table_complex = tf_table_complex["gene_A"].str.split(r"\+")
    

    #tf_complex_receptor_interactions = pd.DataFrame()
    #for complex in tf_table_complex["gene_A"]:
    #  c_receptors = complex.split("+")
    #  print("djspdjog", c_receptors)

     # complex_is_in_lr = list(set(c_receptors) & set(lr_receptors))
     # print("isin", complex_is_in_lr)

    c_receptors = tf_table_receptors[tf_table_receptors["gene_A"].apply(lambda x: any(gene in x.split("+") for gene in lr_receptors))]
    print(c_receptors)
    #tf_complex_receptor_interactions = pd.concat([tf_complex_receptor_interactions, c_receptors])
   
    #print("tf_complex_receptor_interactions", tf_complex_receptor_interactions)
    

In [ ]:
import pandas as pd
import numpy as np
import re

def add_entry(source, target, gene_A, gene_B, type_gene_A, type_gene_B, MeanLR):
    return {
        "source": source,
        "target": target,
        "gene_A": gene_A,
        "gene_B": gene_B,
        "type_gene_A": type_gene_A,
        "type_gene_B": type_gene_B,
        "MeanLR": MeanLR
    }

def generate_CrossTalkeR_input(tf_activities, gene_expression, out_path, regulon=None, organism="human"):
    if organism == "human":
        ligands = pd.read_csv("ligands_human.csv")
        R2TF = pd.read_csv("rtf_db_human.csv")
    elif organism == "mouse":
        ligands = pd.read_csv("ligands_mouse.csv")
        R2TF = pd.read_csv("rtf_db_mouse.csv")
    else:
        raise NameError("Invalid organism to generate CrossTalkeR input!")
    
    # Aggregating regulon data (grouping targets per TF)
    sorted_regulon = regulon.groupby("tf")["target"].apply(list).to_dict()
    
    # Aggregating receptor-TF interactions (grouping receptors per TF)
    R2TF = R2TF.groupby("tf")["receptor"].apply(list).to_dict()
    
    output_list = []
    
    for row in range(len(tf_activities)):
        tf = str(tf_activities["gene"].iloc[row])
        
        # Get targets regulated by the TF
        targets = sorted_regulon.get(tf, [])
        
        # Get receptors associated with the TF
        receptors = R2TF.get(tf, [])
        
        # Identify TF ligands
        tf_ligands = np.intersect1d(targets, ligands)
        
        if organism == "human":
            if len(tf_ligands) > 0:
                for ligand in tf_ligands:
                    expressed = False
                    if ligand in gene_expression.index:
                        ex_value = gene_expression.loc[ligand, tf_activities.iloc[row, 2]]
                        if ex_value != 0:
                            expressed = True
                    
                    if expressed:
                        output_list.append(add_entry(
                            source=tf_activities.iloc[row, 2],
                            target=tf_activities.iloc[row, 2],
                            gene_A=tf_activities.iloc[row, 0],
                            gene_B=ligand,
                            type_gene_A="Transcription_Factor",
                            type_gene_B="Ligand",
                            MeanLR=tf_activities.iloc[row, 3]
                        ))
            
            if len(receptors) > 0:
                for receptor in receptors:
                    output_list.append(add_entry(
                        source=tf_activities.iloc[row, 2],
                        target=tf_activities.iloc[row, 2],
                        gene_A=receptor,
                        gene_B=tf_activities.iloc[row, 0],
                        type_gene_A="Receptor",
                        type_gene_B="Transcription Factor",
                        MeanLR=tf_activities.iloc[row, 3]
                    ))
        else:
            if len(tf_ligands) > 0:
                for ligand in tf_ligands:
                    expressed = False
                    translations = ligand
                    for l in translations:
                        if l in gene_expression.index:
                            ex_value = gene_expression.loc[l, tf_activities.iloc[row, 2]]
                            if ex_value != 0:
                                expressed = True
                    
                    if expressed:
                        output_list.append(add_entry(
                            source=tf_activities.iloc[row, 2],
                            target=tf_activities.iloc[row, 2],
                            gene_A=tf_activities.iloc[row, 0],
                            gene_B=ligand,
                            type_gene_A="Transcription_Factor",
                            type_gene_B="Ligand",
                            MeanLR=tf_activities.iloc[row, 3]
                        ))
            
            if len(receptors) > 0:
                for receptor in receptors:
                    output_list.append(add_entry(
                        source=tf_activities.iloc[row, 2],
                        target=tf_activities.iloc[row, 2],
                        gene_A=receptor,
                        gene_B=tf_activities.iloc[row, 0],
                        type_gene_A="Receptor",
                        type_gene_B="Transcription Factor",
                        MeanLR=tf_activities.iloc[row, 3]
                    ))
    
    # Create output dataframe
    output_df = pd.DataFrame(output_list)
    output_df["gene_A"] = output_df["gene_A"].apply(lambda x: re.sub("_", "+", x))
    output_df["gene_B"] = output_df["gene_B"].apply(lambda x: re.sub("_", "+", x))
    
    # Save output
    output_df.to_csv(out_path, index=False)
    
    return output_df
